# Basic code
## Abstract
- 本チュートリアルで使用する点群のフォーマットについて紹介します。
- また、チュートリアルで頻繁に使用する関数について紹介します。

## TL;DR
- 本チュートリアルでは2次元配列(点の数, チャンネル数)の点群フォーマットを使用します。
- よく使用する関数として、tutlibs.io.Points(読み込み書き込み)とtutlibs.visualization.Points(視覚化)があります。

## Introduction
本チュートリアルでは、点群を2次元配列(点の数, チャンネル数)のフォーマットで処理します。このチャンネル数には、座標情報XYZ(3次元)や色情報RGB(3次元)などの次元数が含まれます。また、点群の出力を確認したり、点群を読み込んだりするための頻繁に使う関数について簡単に紹介します。

以下のsubsectionで実際にこのフォーマットや関数を用いた点群のコードを示します。






## Format
本subsectionでは、点群を2次元配列のフォーマットに従って表します。このsubsectionで使うパッケージは以下のとおりです。

In [3]:
import numpy as np

はじめに、各点の座標が(1, 0, 0)、(0, 0.5, 1)、(1, 1, 0.1)、(-1, 0.5, -0.3)の点群を作成します。

In [4]:
xyz = np.array([
    [1, 0, 0],
    [0, 0.5, 1],
    [1, 1, 0.1],
    [-1, 0.5, -0.3]
], dtype=np.float32)

print('XYZ shape:{}'.format(xyz.shape))


XYZ shape:(4, 3)


コードでは点群の座標情報を`xyz`に入れ、`xyz.shape`で座標情報の形状が(4, 3)となることを確認しました。この(4, 3)は(点の数, 空間の次元数)を示しており、本チュートリアルでは3次元空間(XYZ)上の点群を扱うため、空間の次元数=3となっています。

Note: 本チュートリアルでは3次元空間上の点群を利用しますが、取得するセンサーなどによっては2次元空間上の点群などもあります(Turtlebot3の[LSD-01](https://emanual.robotis.com/docs/en/platform/turtlebot3/appendix_lds_01/)など)。その場合は(点の数, 2)として扱うか、(点の数, 3)で使わない次元に0を代入するといった方法を取ります。


点群データには座標値以外に色情報などの追加情報が含まれていることもあります。上記のコードの各点に色が割り振られている場合、色情報は以下の様に表されます。

In [5]:
rgb = np.array([
    [255, 10, 20],
    [0, 89, 111],
    [74, 116, 121],
    [190, 95, 3]
], dtype=np.int32)

print('RGB shape:{}'.format(rgb.shape))

RGB shape:(4, 3)


色情報`rgb`の形状は(4, 3)となり、この形状は(点の数, 色情報の次元)を示しています。また、`rgb`のi番目の色(`rgb[i]`)は`xyz`のi番目の座標点(`xyz[i]`)と対応しています。  

ここまでの座標情報や色情報より、チュートリアル上で点群データを(点の数, チャンネル数(チャンネルは座標値、RGBなど))という形で扱っていることが分かると思います。この(点の数, チャンネル数)のフォーマットは、実際の点群処理で扱われる基礎的なフォーマットです。本リュートリアルでもこのシンプルなフォーマットを主に使用し、点群処理についての紹介を行います。

前述したとおり、本チュートリアルでは上記の点群データフォーマットを使用します。また、チュートリアルにおいて点群処理をわかりやすくするため、点群データに対して取り決めを追加します。以上より、チュートリアルの仕様は以下の通りです。

- データは(点の数, チャンネル数)の形状を取る。
- データは色や座標など、情報の種類にごとに変数を分けて扱う。
- 点群中のi番目の点情報は`変数[i]`で取得できる(xyz[i], rgb[i]など)。


Note: この点群のデータ形状は以下のように示すことができます: 点群の座標データ点群を$P$、座標値を持つ点を$p=(x, y, z)$として表す時、$N$個の点を持つ$P$は以下の通り。

$$
\begin{array}{l l}
P = [ \\
& [x_1,\ y_1,\ z_1] \\
& [x_2,\ y_2,\ z_2], \\
& \ldots , \\
& [x_n,\ y_n,\ z_n], \\
& \ldots , \\
& [x_N,\ y_N,\ z_N] \\
] \\
\end{array}
$$


Note: 本チュートリアルでは(点の数, チャンネル数)を使用します。これ以外にも、画像ライクな形状である(width, height, channel size)があります。こちらは、[PCL](https://pcl.readthedocs.io/projects/tutorials/en/latest/basic_structures.html#basic-structures)で設けられています。また形状以外にも、チャンネルの取扱方法として一つの変数に各チャンネルをまとめる方法が採用されている場合もあります。例えば、[Open3D](http://www.open3d.org/docs/0.12.0/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.colors)は一つの点群の型に複数のチャンネルが格納されており、これらのチャンネル情報はクラス変数を介して各チャンネルにアクセスできるように設計されています。

## Function
このsubsectionでは、チュートリアルで頻繁に使用する関数について説明します。説明対象となる関数は以下のとおりです。

- 点群ファイルの読み込みと書き込み (tutlibs.io.Points as io)
- 点群の視覚化 (tutlibs.visualization.Points as visualizer)

このsubsectionで使用するパッケージは以下のとおりです。


In [6]:
import numpy as np
import os
from tutlibs.visualization import Points as visualizer
from tutlibs.io import Points as io

### 点群ファイルの読み込みと書き込み (tutlibs.io.Points as io)
はじめに、点群ファイルの点群を`io.read`で読み取ります。コードは以下の通りです。

In [7]:
xyz, rgb, data = io.read('../data/bunny_pc.ply')
print('XYZ shape:{}'.format(xyz.shape))
print('RGB shape:{}'.format(rgb.shape))
print('data type:{}, key:{}'.format(type(data), data.keys()))

XYZ shape:(3000, 3)
RGB shape:(3000, 3)
data type:<class 'dict'>, key:dict_keys(['nx', 'ny', 'nz'])


`io.read`は`ply`ファイルからデータを読み取ります。注意点として、plyファイルはメッシュデータを取り扱うファイルですが、メッシュデータの頂点を点群とみなすことで、ここではplyファイルを点群ファイルの様に取り扱っています。
`io.read`の返り値は(`xyz`, `rgb`, `data`)の3つです。返り値の説明は以下のとおりです。
1. `xyz`: 点群の座標情報が入ります。この関数ではYXZ値を取得するため、受け取る値は(点の数, 3)となります。
2. `rgb`: 点群の色情報が入ります。この関数ではファイル内のRGB値を色情報として読み込むため、色情報がある場合は(点の数, 3)を返します。色情報がない場合はNoneを返します。
3. `data`: ファイル内のデータを辞書形式で保存します。一部sectionでのみ使用します。

次に、この点群を`io.write`でplyファイルに保存します。コードは以下の通りです。

In [8]:
os.makedirs('outputs/',exist_ok=True)
io.write('outputs/basic_code_pc.ply', xyz, rgb, color_range=[0, 255],
         additional_data=data)

`io.write`は`ply`ファイルへ点群を保存します。保存する際に必要となる値は以下のとおりです。
- `filename`: 保存ファイルパス
- `xyz`: 点群の座標情報, 入力配列の形状は(点の数, 3)となります。
- `rgb`: 点群の色情報, 入力配列の形状は(点の数, 3)となります。初期値はNoneであり、入力がNoneの場合は色情報を保存しません。
- `color_range`: 色情報の値の範囲を入力します。初期値は[0, 1]であり、これは入力した色情報(RGB値)が0~1の範囲で示されている場合の設定となります。もしRGB値の範囲が0~255の範囲である場合は、[0, 255]となります。
- `additional_data`: 追加で保存するデータを辞書形式で入力します。初期値はNoneであり、Noneの場合は追加の情報は入力されません。基本は使いません。


### 点群の視覚化 (tutlibs.visualization.Points as visualizer)
最後に、変数の点群を視覚化します。コードは以下のとおりです。

In [9]:
visualizer.k3d(xyz, rgb, color_range=[0, 255])

Output()

`visualizer.k3d`はjupyter上で点群を視覚化します。引数は以下のとおりです。
- `xyz`: 視覚化する点群の座標情報, 入力配列の形状は(点の数, 3)となります。
- `rgb`: 視覚化する点群の色情報, 入力配列の形状は(点の数, 3)となります。初期値はNoneであり、入力がNoneの場合は色情報を反映せず、点群を青で表示します。
- `color_range`: 色情報の値の範囲を入力します。初期値は[0, 1]であり、これは入力した色情報(RGB値)が0~1の範囲で示されている場合の設定となります。もしRGB値の範囲が0~255の範囲である場合は、[0, 255]となります。

visualizer.k3dには他にも引数がありますが、ここでは省略します。

## 開発に関するTopic
### TL;DL;
- 点群のコードを書く場合は、型とデータ形状のヒントを載せることを推奨します。
- 本チュートリアルでは、コメントと[typing](https://docs.python.org/ja/3/library/typing.html)によるヒントを使います。

### 説明
上記では点群のチャンネルとして座標値と色情報を取り扱いましたが、点群ではこれら以外のチャンネルを扱うこともあります。例えば、LiDARと呼ばれるセンサーからは反射強度を得ることがあります。また、他の3D表現データから点群をサンプリングした場合、法線情報も取得できる場合があります。  
また、チャンネルの変化以外にも、処理の過程でデータの形状がいくらか変化することが多いです。例えば、kNNによって点ごとの近傍を得る場合、データの形状は(点の数, kNNの近傍数, チャンネル数)といった形に変化します。

この様に、点群データの変数はいろいろと変化します。この変化は共同開発や再利用時にコードの理解を妨げる要因になりやすいです。従って、実際に点群データを使用するコードを書く場合は、関数の引数や返り値などに[通常のヒント](https://docs.python.org/3/library/typing.html)と形状に関するヒントを載せておくことを強く推奨します。ここの「形状に関するヒント」とは、(点の数, チャンネル数)のことを指します。

これらのヒントはコメントや[typing](https://docs.python.org/ja/3/library/typing.html)以外にも、[numpy typing](https://numpy.org/devdocs/reference/typing.html)、[torchtyping](https://github.com/patrick-kidger/torchtyping)で示すことができます。ただし、2021/09/05現在、本チュートリアルではコメントと[typing](https://docs.python.org/ja/3/library/typing.html)によるヒントの実装を行います。理由としては、
- 実装者本人の慣れ
- 実装時期(typing以外が新しすぎる)

In [ ]:
# for debug
%load_ext autoreload
%autoreload 2